Crawling

Scraping - content - type; text/html, application/json, /image/*, ...
                        => Focused crawling + scraping

Dynamic HTML (AJAX - XmlHttpRequest/JS/Doc...)
            => DOM (Req/Resp), Data(Req/Resp => content-type), 예시로서 웹툰

Cookie / Session

                HTTP
        Client              Server
        Req     =========>  login, 회원가입, ... => REST API (Put, Post)
        Resp    <=========  HTML Form (method, action / JS) -> 
                        변수 = 우리 정보
        Cookie(*)           Session
                (key)

In [3]:
from requests import request
from requests.compat import *
from bs4 import BeautifulSoup

headers = {
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [5]:
url = 'https://pythonscraping.com/pages/cookies/login.html'
resp = request('GET', url)

In [6]:
resp.status_code

200

In [7]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [8]:
form = dom.select_one('form')

In [9]:
form.attrs['action']

'welcome.php'

In [10]:
[tag.attrs['name'] for tag in form.select('[name]')]

['username', 'password']

In [11]:
urljoin(resp.url, form.attrs['action'])

'https://pythonscraping.com/pages/cookies/welcome.php'

In [14]:
params = {
    'username' : 'testest',
    'password' : 'password'
}

resp = request(form.attrs['method'], urljoin(resp.url, form.attrs['action']), data = params)

In [15]:
resp.headers['set-cookie']

'loggedin=1, username=testest'

In [18]:
request('GET', urljoin(resp.url, 'profile.php')).text

'You\'re not logged into the site!<br>Visit <a href="login.html">the login page</a> to log in'

In [19]:
request('GET', urljoin(resp.url, 'profile.php'), cookies={'loggedin' : '1', 'username' : 'test'}).text

"Hey test! Looks like you're still logged into the site!"

In [21]:
resp.request.url, resp.request.body

('https://pythonscraping.com/pages/cookies/welcome.php',
 'username=testest&password=password')

In [22]:
from requests.cookies import cookiejar_from_dict

In [26]:
c = cookiejar_from_dict({'loggedin' : '1', 'username' : 'whoami'})

In [27]:
request('GET', urljoin(resp.url, 'profile.php'), cookies=c).text

"Hey whoami! Looks like you're still logged into the site!"

In [28]:
for row in resp.headers['set-cookie'].split(','):
    print(row.strip().split('='))

['loggedin', '1']
['username', 'testest']


In [ ]:
resp = session.request(form.attrs['method'], urljoin(resp.url, form.attrs['action']), data = params)

In [ ]:
session.request('GET', urljoin(resp.url, 'profile.php')).text

In [29]:
url = 'https://lms.sunde41.net'
resp = request('GET', url)

In [30]:
resp.status_code

200

In [31]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [33]:
len(dom.select('form'))

1

In [36]:
params = {}

for tag in form.select('[name]'):
    print({tag.attrs['name']:tag.attrs['value'] if tag.has_attr('value') else None})

{'username': None}
{'password': None}


In [38]:
params = {}

for tag in form.select('[name]'):
    params[tag.attrs['name']] = tag.attrs['value'] if tag.has_attr('value') else ''

In [44]:
%%writefile lms.txt
{
    "id" : "unijellytwo@naver.com",
    "pw" : "01077034572"
}

Overwriting lms.txt


In [45]:
import json

In [46]:
with open('lms.txt', 'r', encoding = 'utf-8') as fp:
    lms = json.load(fp)

In [47]:
from requests import session

In [ ]:
session.cookies.clear()

In [49]:
form.attrs

{'method': 'post', 'action': 'welcome.php'}

In [ ]:
resp = session.request(form.attrs['method'], urljoin(resp.url, form.attrs['action']), data = params)

In [ ]:
session.request('GET', url)

In [ ]:
dom.select('a[href]^="/course"]')

In [ ]:
resp = request('GET', urljoin(url, '/course'))

In [ ]:
c = {}
for k,v in session.cookies.get_dict().items():
    c[k] = v

In [52]:
request('GET', url, cookies=c).text

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, shrink-to-fit=no">\n    <meta name="description" content="">\n    <meta name="author" content="">\n    <title> 로그인 : 고려대학교 지능정보SW아카데미</title>\n    <link href="/static/vendors/login.css" rel="stylesheet">\n    <link href="/static/vendors/bootstrap.css" rel="stylesheet">\n    <link href="/static/vendors/style.css?v=1.7" rel="stylesheet">\n    <link rel="shortcut icon" href="/static/favicon/favicon.ico" type="image/x-icon">\n    <link rel="icon" href="/static/favicon/favicon.ico" type="image/x-icon">\n    <style>\n        .m-form .m-form__group {\n        padding-top: 0;\n        padding-bottom: 0;\n        }\n    </style>\n</head>\n<body>\n    <div class="form-body without-side">\n        <div class="row">\n            <div class="img-holder">\n                <d

auto/login -> /lms.sunde41.net -> dashboard
            302                 302

In [53]:
from requests.sessions import Session
session = Session()

In [54]:
resp = request('POST', urljoin(url, '/auth/login'), data=params)

In [61]:
url = 'https://nid.naver.com/nidlogin.login?mode=form'

resp = request('GET', url)

In [62]:
session.cookies.clear()

In [63]:
dom = BeautifulSoup(resp.text, 'lxml')
dom.select_one('form').select('[name]')

[<input id="localechange" name="localechange" type="hidden" value=""/>,
 <input id="dynamicKey" name="dynamicKey" type="hidden" value="kVVrx4CpVGBtkp_EvbtdK1ktHOR7bhnzggr1hOpAO5QXhxFa9I6isCsnkR_zngyNOa_yTBUC1wq_dzJQiE_2Rit5f9Egs--nhD_k1CBWeRM"/>,
 <input id="encpw" name="encpw" type="hidden" value=""/>,
 <input id="enctp" name="enctp" type="hidden" value="1"/>,
 <input id="svctype" name="svctype" type="hidden" value="1"/>,
 <input id="smart_LEVEL" name="smart_LEVEL" type="hidden" value="1"/>,
 <input id="bvsd" name="bvsd" type="hidden" value=""/>,
 <input id="encnm" name="encnm" type="hidden" value=""/>,
 <input id="locale" name="locale" type="hidden" value="ko_KR"/>,
 <input id="url" name="url" type="hidden" value="https://www.naver.com"/>,
 <input class="input_text" id="id" maxlength="41" name="id" placeholder="아이디" title="아이디" type="text" value=""/>,
 <input class="input_text" id="pw" maxlength="16" name="pw" placeholder="비밀번호" title="비밀번호" type="password"/>,
 <input class="input_ke

In [59]:
len(dom.select('form'))

1

In [ ]:
dom = BeautifulSoup(resp.text, 'lxml')
dom.select_one('form').select('[name]')

In [64]:
r = '''
JSESSIONID	F59509BADECC255288BE081984ADF26F	www.naver.com	/	Session	42	✓				Medium	
tooltip_paging_close	1	www.naver.com	/	2024-07-07T07:29:50.729Z	21					Medium	
NID_JKL	WIsD5YPdvpXCNo1w5W3sOceFvJF2uOe8xUczxNFiFOU=	.naver.com	/	Session	51		✓			Medium	
nx_ssl	2	.naver.com	/	2023-08-11T07:52:53.558Z	7					Medium	
MY_NUDGE	1	www.naver.com	/	2024-05-06T04:53:45.000Z	9					Medium	
NID_SES	AAABj6UxLfBN6t+D4rJBngQ9qP7yeuo7av/WnNmL2c3sJutOsvLDf6Z7+H/iVJllqEXIE/rat7ITpHqU0pyPSy3UQe5gStbubkk2H0z1cUkdoavqKwvmFK4jFp3YxcVigJUMWtCJ2qoZGmvlFEBh9sY13Pjy7x70pl1ytgYOOQRL47+T4/kLtubIUGsx/cbukL491pxQZyWoWWxDqRGLdWB56FqNNuvLFLlSfVUIeyJXtsFIuLh0NZVB89I3MTA4b9t1bGS2rYtzqrOatezcC51PyC2CA/ak6hnghM4mxgYGIh4aavVevNRa1mpz1fwzTBELNUN+5H/XiesjRQU2Vww3AegIJxZ5pxC/o6fN2kJ3JQq+lbFwOtCPVi6ooiFqk3W4qjbaud9H5zBgtJgrhYpJuwZoyOWhU9KNzn0ClXFfGYpf3aVyPFpL9aDfLHLUhUxA4J2OFI3g2xmg9Su9pUuu/zTaRL3w5OMmSlf5stT3PzLhJ3BmtKz0AXlDVEKbaOjBQvnA9LNiPLm+4+x2oouax24=	.naver.com	/	Session	547					Medium	
NID_AUT	hote1BBUvn0zhd891GwYqijUAZaoskPrCCKv+grbmY9xSrwGYycF7XFVSXuPvBYH	.naver.com	/	Session	71	✓				Medium	
page_uid	i7fohdp0YihssTpSd/dssssstYR-073381	.naver.com	/	Session	42					Medium	
nid_inf	-1269507578	.naver.com	/	Session	18					Medium	
NNB	4FWLQHW2JUWWI	.naver.com	/	2024-08-20T02:40:03.665Z	16		✓	None		Medium	
nid_slevel	1	.nid.naver.com	/	2024-07-16T02:39:58.000Z	11					Medium	
PM_CK_loc	2b6720e78c216d0746bf1f8ec13e12645283961615268c1cf9cf2c843f5c4d5e	www.naver.com	/	2023-07-18T02:22:33.094Z	73	✓				Medium	
ASID	dee9324300000187760bd83900000049	.naver.com	/	2024-07-11T07:18:26.920Z	36		✓	None		Medium	
nid_buk	4FWLQHW2JUWWI	.nid.naver.com	/	2024-07-11T06:42:42.575Z	20		✓			Medium	
'''
c = {}
for line in r.splitlines():
    if len(line) > 1:
       # print(line.split('\t')[:2])
       c[line.split('\t')[0]] = line.split('\t')[1]

In [67]:
url = 'https://mail.naver.com'
request('GET', url, cookies=c).text


'<!doctype html><html><head lang="ko"><meta charset="UTF-8"/><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,viewport-fit=cover"/><title>메일</title><link rel="shortcut icon" href="/favicon.ico" type="image/x-icon"/><link rel="apple-touch-icon-precomposed" sizes="72x72" href="https://ssl.pstatic.net/static/pwe/naver_mail/hdpi_72X72.png"><link rel="apple-touch-icon-precomposed" sizes="96x96" href="https://ssl.pstatic.net/static/pwe/naver_mail/xhdpi_96x96.png"><link rel="apple-touch-icon-precomposed" sizes="144x144" href="https://ssl.pstatic.net/static/pwe/naver_mail/xxhpdi_144x144.png"><link rel="apple-touch-icon-precomposed" sizes="192x192" href="https://ssl.pstatic.net/static/pwe/naver_mail/xxxhpdi_192x192.png"><script>window.nhn = {};\n      window.gnb_option = {\n        gnb_service: "mail",\n        gnb_template: "gnb_utf8",\n        gnb_logout: "https://www.naver.com",\n        gnb_one_naver: 1,\n        gnb_item_hide_option: 8,\n   

In [69]:
for k,v in c.items():
    session.cookies.set(k,v)

In [70]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='ASID', value='dee9324300000187760bd83900000049', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='JSESSIONID', value='F59509BADECC255288BE081984ADF26F', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='MY_NUDGE', value='1', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_AUT', value='hote1BBUvn0zhd891GwYqi

In [71]:
import re

In [72]:
re.search(r'<title>(.+)</title>', session.request('GET', url).text)

<re.Match object; span=(179, 196), match='<title>메일</title>'>

In [74]:
session.get('https://mail.naver.com/v2/folders/0').text

'<!doctype html><html><head lang="ko"><meta charset="UTF-8"/><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,viewport-fit=cover"/><title>메일</title><link rel="shortcut icon" href="/favicon.ico" type="image/x-icon"/><link rel="apple-touch-icon-precomposed" sizes="72x72" href="https://ssl.pstatic.net/static/pwe/naver_mail/hdpi_72X72.png"><link rel="apple-touch-icon-precomposed" sizes="96x96" href="https://ssl.pstatic.net/static/pwe/naver_mail/xhdpi_96x96.png"><link rel="apple-touch-icon-precomposed" sizes="144x144" href="https://ssl.pstatic.net/static/pwe/naver_mail/xxhpdi_144x144.png"><link rel="apple-touch-icon-precomposed" sizes="192x192" href="https://ssl.pstatic.net/static/pwe/naver_mail/xxxhpdi_192x192.png"><script>window.nhn = {};\n      window.gnb_option = {\n        gnb_service: "mail",\n        gnb_template: "gnb_utf8",\n        gnb_logout: "https://www.naver.com",\n        gnb_one_naver: 1,\n        gnb_item_hide_option: 8,\n   

In [76]:
url = 'https://mail.naver.com/json/list'

params = {
    'folderSN':'0',
    'page':'1',
    'viewMode':'time',
    'previewMode':'1',
    'sortField':'1',
    'sortType':'0',
    'u':'unijellytwo'
}

resp = session.request('POST', url, params=params)

In [77]:
len(resp.json()['mailData'])

30

In [79]:
params = {
    'folderSN':'5',
    'page':'1',
    'viewMode':'time',
    'previewMode':'1',
    'sortField':'1',
    'sortType':'0',
    'u':'unijellytwo'
}

In [80]:
resp = request('POST', url, params=params, cookies=c)
len(resp.json()['mailData'])

0

In [81]:
resp = session.request('GET', 'https://cafe.naver.com')

In [82]:
url = 'https://apis.naver.com/cafe-home-web/cafe-home/v1/homepc'

In [83]:
params = {
    'page' : '1',
    'type' : 'join',
    'perPage' : '10',
    'recentUpdates' : 'true'
}

In [84]:
resp = request('GET', url, params=params, cookies=c)


In [98]:
len(resp.json())

1

In [100]:
url = 'https://www.ypbooks.co.kr/kor_index.yp'

resp = request('GET', url)

dom = BeautifulSoup(resp.text, 'lxml')

In [101]:
dom.select('form')[4]

<form action="https://www.ypbooks.co.kr/search.yp?targetpage=search_total_main" method="post" name="ypsearch">
<input name="query" type="hidden" value=""/>
<input name="searchField" type="hidden" value=""/>
<input name="sortField" type="hidden" value="RANK"/>
<input name="order" type="hidden" value="1"/>
<input name="idx" type="hidden" value="0"/>
<input name="rowNum" type="hidden" value="20"/>
<input name="notSoldOut" type="hidden" value="Y"/>
<input name="c1" type="hidden" value=""/>
<input name="c2" type="hidden" value=""/>
<input name="c3" type="hidden" value=""/>
<!-- 상세검색용 -->
<input name="detailchk" type="hidden" value=""/>
<input name="d_title" type="hidden" value=""/>
<input name="d_author" type="hidden" value=""/>
<input name="d_cusnme" type="hidden" value=""/>
<input name="d_body" type="hidden" value=""/>
<input name="d_isbn" type="hidden" value=""/>
<input name="s_price" type="hidden" value=""/>
<input name="e_price" type="hidden" value=""/>
<input name="pubdt" type="hidden

In [110]:
form = dom.select('form')[-1]

params = {node.attrs['name'] : node.attrs['value'] for node in form.select('[name]')}

In [111]:
params['query'] = '뉴진스'

In [114]:
resp = request(form.attrs['method'], urljoin(url, form.attrs['action']), data=params, headers=headers['user-agent'])

In [115]:
form.attrs

{'name': 'searchFrm', 'method': 'post', 'action': '/search.yp'}

In [118]:
resp.status_code, resp.reason, resp.headers

(200,
 '200',
 {'Date': 'Mon, 17 Jul 2023 03:13:16 GMT', 'Set-Cookie': 'SCOUTER=x263fkldv58k97; Max-Age=2147483647; Expires=Sat, 04-Aug-2091 06:27:23 GMT; Path=/, JSESSIONID=856AB6DBD01F39CCFC74A8EDA5FBD67B; Path=/; Secure; HttpOnly', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html;charset=UTF-8'})

In [117]:
print(resp.text)




















<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
	<title>영풍문고 - 서점다운 서점</title>
	<meta http-equiv="X-UA-Compatible" content="IE=edge" />
	<link rel="shortcut icon" href="https://img.ypbooks.co.kr/ypbooks/images/new_favicon.ico" type="image/x-icon" />
		<style type="text/css">
		#allFrame {width:/*1080px*/100%;margin:0px auto;position:relative;overflow:hidden;}
		#mainFrame {width:1080px;margin:0px auto;position:relative;}
		#allFrame #cartArea {width:450px;;height:500px;right:0px;position:absolute;text-align:right;z-index:1000;top:192px;}
		#mainFrame #cartArea {width:450px;;height:500px;right:0px;position:absolute;text-align:right;z-index:1000;top:144px;}
		</style>
		<script type="text/javascript" src="/ypbooks/js/jquery-1.12.4.min.js"></script>
		<script type="text/javascript" src="/ypbooks/js/jquery.bgiframe.min.js"></script>
		<script type=

In [ ]:
# 인스타그램
url = 'https://www.instagram.com/api/v1/web/accounts/login/ajax'

params = {

}

resp = request('GET', url, data=params)
resp.text

In [122]:
from datetime import datetime

In [123]:
datetime.now().timestamp()

1689566534.513274

In [124]:
url = "https://www.instagram.com"
resp = request('GET', url)

In [ ]:
re.search(r'')